In [14]:
import time
import selenium.webdriver as webdriver
from bs4 import BeautifulSoup
from cffi.cffi_opcode import CLASS_NAME
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pymysql

In [15]:
# db에 저장하는 함수
def save_mariadb(dict_list):
    db = pymysql.connect(host='localhost',user='webuser',passwd='webuser',db='webdb',charset='utf8')
    cursor = db.cursor()
    query = "INSERT INTO busanAttraction (name, score, img) VALUES (%s, %s, %s)"

    try:
        # tuple 로 들어가야함, 변환
        cursor.executemany(query, [(d['name'], d['score'], d['img']) for d in dict_list])
        db.commit()
        print(f"db에 저장 완료")
    except Exception as e:
        db.rollback()
        print(f"에러 발생 {e}")
    cursor.close()
    db.close()

In [16]:
wdriver = webdriver.Chrome()
wdriver.get("https://kr.trip.com/?locale=ko-KR&curr=KRW")
wdriver.maximize_window()

guide_category = WebDriverWait(wdriver,10).until(
    EC.element_to_be_clickable((By.ID, 'header_action_nav_travelguide'))
)
guide_category.click()
search_input = WebDriverWait(wdriver,10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="여행지, 명소, 호텔 등으로 검색"]'))
)
search_input.send_keys("부산")
search_input.send_keys(Keys.ENTER)
busan_selector = WebDriverWait(wdriver,10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href="https://kr.trip.com/travel-guide/destination/busan-432/?locale=ko-KR&curr=KRW"]'))
)
busan_selector.click()

busan_attraction_selector = WebDriverWait(wdriver,10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-exposure-traceid="tgs_dstdetail_expo_poirank_more"]'))
)
wdriver.execute_script("arguments[0].click();",busan_attraction_selector)

# 가고싶은 페이지까지 도달했다면
html = wdriver.page_source
soup = BeautifulSoup(html, 'html.parser')

attraction_list = []
attraction_tag_list = soup.find_all("li", {"class": 'sc-jtcaXd lMYZD burited_point'})
for idx, attraction in enumerate(attraction_tag_list):
    if idx > 9:
        break
    # 여기는 태그
    attraction_img = attraction.find("img", {"class": 'sc-cLFqLo foPZxf'})
    attraction_name = attraction.find("a", attrs={"title":True})
    attraction_score = attraction.find("span", {"class" : 'hotScore'})
    print(f"이름:{attraction_name['title'][:-6]} 사진src:{attraction_img['src']} 점수:{attraction_score.text}")
    # 메모리에 추가
    attraction_list.append({"name": attraction_name['title'][:-6], "score": attraction_score.text, "img": attraction_img['src']})

time.sleep(3)
save_mariadb(attraction_list)
wdriver.quit()
# 위의 사이트에서 즐길거리 카테고리 클릭, 즐길거리&랜드마크 버튼 클릭
# 사진, 이름, 별점? or 리뷰수 저장
# 다음 버튼 클릭
# 반복

이름:해운대블루라인파크 사진src:https://ak-d.tripcdn.com/images/1lo2d12000eh5t1a9E757_C_500_300_R5.jpg_.webp?proc=source%2ftrip 점수:7.6
이름:송도해상케이블카 사진src:https://ak-d.tripcdn.com/images/0100g12000876zhn6414E_C_500_300_R5.jpg_.webp?proc=source%2ftrip 점수:7.1
이름:부산 엑스 더 스카이 사진src:https://ak-d.tripcdn.com/images/1lo6t12000c4sutqlDF78_C_500_300_R5.jpg_.webp?proc=source%2ftrip 점수:6.8
이름:감천문화마을 사진src:https://ak-d.tripcdn.com/images/1A07170000011rv1964E6_C_500_300_R5.jpg_.webp?proc=source%2ftrip 점수:6.5
이름:롯데월드 어드벤처 부산 사진src:https://ak-d.tripcdn.com/images/1lo0c12000hyihoz39AD4_C_500_300_R5.png_.webp?proc=source%2ftrip 점수:6.4
이름:씨라이프 부산 아쿠아리움 사진src:https://ak-d.tripcdn.com/images/1lo6412000g74boxdF1E0_C_500_300_R5.jpg_.webp?proc=source%2ftrip 점수:6.3
이름:해동용궁사 사진src:https://ak-d.tripcdn.com/images/10051f000001gsri99F3C_C_500_300_R5.jpg_.webp?proc=source%2ftrip 점수:6.2
이름:해운대해수욕장 사진src:https://ak-d.tripcdn.com/images/fd/tg/g4/M02/54/BB/CggYHlaUzoeAXGcTACOEW7lnhZU348_C_500_300_R5.jpg_.webp?proc=source%2ftrip 점수:6